In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import math
data = pd.read_csv('all/train.csv')
testdata = pd.read_csv('all/test.csv')
attributes = ["penalty"]
#print(data)
for i in attributes:
    a = preprocessing.LabelEncoder()
    a.fit(list(data[i].values.astype('str')))
    data[i] = a.transform(list(data[i].values.astype('str')))   
#print(data)
#create a python list of feature names
data=data[(True-data['n_jobs'].isin([-1]))]   #drop the raw which n_jobs are -1
feature_cols = [ 'penalty', 'l1_ratio', 'alpha', 'random_state', 'n_classes', 'n_clusters_per_class', 'flip_y', 'scale', 'new', 'new2', 'new3', 'new4', 'new5']
# use the list to select a subset of the original DataFrame
data['new']=data['max_iter']*data['n_features']*data['n_samples']/data['n_jobs']
data['new2'] =data['n_features']* data['max_iter']
data['new3'] = data['max_iter']/data['n_jobs']
data['new4'] = data['n_samples']/data['n_jobs']
data['new5'] = data['n_features']/data['n_jobs']
data['alpha']=data['alpha'].replace(0.001,1)
data['alpha']=data['alpha'].replace(0.01,2)
data['alpha']=data['alpha'].replace(0.0001,0)
X = data[feature_cols]
X
# print the first 5 rows
#print(X)
# check the type and shape of X
#print(type(X))
print(X.shape)

(326, 13)


D:\Anaconda\envs\tensorflow\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


In [2]:
for i in attributes:
    a = preprocessing.LabelEncoder()
    a.fit(list(testdata[i].values.astype('str')))
    testdata[i] = a.transform(list(testdata[i].values.astype('str')))   
    
testdata['n_jobs']=testdata['n_jobs'].replace(-1,16)
testdata['new']=testdata['max_iter']*testdata['n_features']*testdata['n_samples']/testdata['n_jobs']
testdata['new2'] =testdata['n_features']* testdata['max_iter']
testdata['new3'] = testdata['max_iter']/testdata['n_jobs']
testdata['new4'] = testdata['n_samples']/testdata['n_jobs']
testdata['new5'] = testdata['n_features']/testdata['n_jobs']
testdata['alpha']=testdata['alpha'].replace(0.001,1)
testdata['alpha']=testdata['alpha'].replace(0.01,2)
testdata['alpha']=testdata['alpha'].replace(0.0001,0)

print(feature_cols)
testX = testdata[feature_cols]
# print the first 5 rows
#print(testdataX)
# check the type and shape of X
#print(type(testdataX))
print(testX.shape)

['penalty', 'l1_ratio', 'alpha', 'random_state', 'n_classes', 'n_clusters_per_class', 'flip_y', 'scale', 'new', 'new2', 'new3', 'new4', 'new5']
(100, 13)


In [3]:
import math
# select a Series from the DataFrame
y =data['time'].map(lambda x: math.log(x))
# print the first 5 values
print(y.shape)

(326,)


In [4]:
import sklearn
X_train=X
y_train=y
X_test=testX
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(326, 13)
(326,)
(100, 13)


In [5]:
import xgboost as xgb
from xgboost import XGBRegressor
bst=XGBRegressor(max_depth=3,min_child_weight=4,learning_rate=0.2,subsample=0.5,colsample_bytree=0.7,gamma=0,n_estimators=400)
bst.fit(X_train, y_train)
y_pred=bst.predict(X_test)
y_pred = [math.exp(c) for c in y_pred]
print(y_pred)

D:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[1.374081327365478, 10.124537270282989, 0.2991311483596629, 0.6370612917898785, 1.970100021644655, 7.418662288880962, 2.78074375465226, 0.38163279076516987, 11.555664402358707, 0.4498171131239051, 6.33542157625818, 8.936883953694215, 0.5064448060818723, 29.031300807380617, 0.2406708924850186, 0.7947566743342582, 0.556518269910223, 5.2003614471961725, 2.7912003437544683, 0.703801156846396, 0.4176354753180718, 0.18766467278697474, 0.4025743365388185, 0.30565188467015053, 1.3949469098551575, 0.9781202307843807, 1.3330830356916161, 1.8408421812300488, 1.7722700061628873, 1.802273712302756, 15.420627958237743, 0.997645769484719, 0.33469408916084775, 8.125749232249035, 7.176390817944996, 0.9627679838943533, 2.4445931257650657, 1.4128938876546033, 0.9307069857066216, 2.0156643321153958, 3.3874931051820316, 10.52834998331514, 0.5617258622000366, 3.8087189904431593, 1.0525854535363885, 3.243606828466784, 2.0713914551523356, 0.38338509704745777, 8.99717697020515, 0.8263325940432086, 0.2040902245

In [6]:
from sklearn import metrics
preds_train = bst.predict(X_train)
metrics.r2_score(y_train.values,preds_train)

0.99997337964095567

In [7]:
y_pred=pd.DataFrame(y_pred)
y_pred.columns=['time']
y_pred.to_csv('test.csv',index_label='Id')